In [9]:
!pip install -q llama-index llama-index-llms-groq  llama-index-embeddings-huggingface   llama-parse

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-ollama 0.4.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-embeddings-instructor 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.


In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [12]:
print(GROQ_API_KEY)

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


In [13]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192", api_key=GROQ_API_KEY)
llm_70b = Groq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY)

AttributeError: 'FieldInfo' object has no attribute 'field_info'

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/opt/anaconda3/envs/COS243/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

NameError: name 'llm' is not defined

In [8]:
LLAMAINDEX_API_KEY = os.getenv('LLAMAINDEX_API_KEY')
print(LLAMAINDEX_API_KEY)

llx-HgVBhgVsv7PiFK5Xs5QD1IFHVuTmIX2le93iDT1ZSYqtFsb3


In [11]:
from llama_index.core import SimpleDirectoryReader
book_path = "./upload/The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code Hunted Down Russian Submarines and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf"

docs_bayes = SimpleDirectoryReader(input_files=[book_path]).load_data()


In [12]:
docs_bayes[2].metadata

{'page_label': '3',
 'file_name': 'The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code Hunted Down Russian Submarines and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf',
 'file_path': 'upload/The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code Hunted Down Russian Submarines and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf',
 'file_type': 'application/pdf',
 'file_size': 2592963,
 'creation_date': '2024-12-10',
 'last_modified_date': '2024-12-10'}

In [48]:
response = llm.complete("do you like Thomas Bayes?")

print(response.text)

I'm just an AI, I don't have personal preferences or emotions, but I can tell you that Thomas Bayes was a British mathematician and Presbyterian minister who lived in the 18th century. He is best known for his work on probability theory, and his famous theorem, Bayes' theorem, is a fundamental concept in statistics and data analysis.

Bayes' theorem is a mathematical formula that describes how to update the probability of a hypothesis based on new evidence. It is widely used in many fields, including medicine, engineering, economics, and computer science.

Many people consider Bayes to be one of the most important mathematicians of the 18th century, and his work has had a significant impact on the development of statistics and probability theory.


In [49]:
stream_response = llm.stream_complete(
    "you're a Bayesian fan. tell me why you like Bayesian more than Frequentists"
)

for t in stream_response:
    print(t.delta, end="")

A delightful topic! As a Bayesian enthusiast, I'm excited to share with you why I prefer the Bayesian approach over the frequentist one. Here are some key reasons:

1. **Interpretation of probability**: Bayesians view probability as a measure of uncertainty or degree of belief, whereas frequentists see it as a long-run relative frequency. I find the Bayesian interpretation more intuitive and flexible, as it allows us to update our beliefs based on new evidence.
2. **Model uncertainty**: Bayesians acknowledge that models are never perfect and that there's always uncertainty about the true underlying process. We can incorporate this uncertainty into our analysis by using hierarchical models, which allow us to model the uncertainty in the model itself. Frequentists, on the other hand, often assume that the model is known with certainty.
3. **Handling missing data**: Bayesians can easily handle missing data by using techniques like multiple imputation or Bayesian data augmentation. This is

In [50]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are a Bayesian fan."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [51]:
print(response)

assistant: "In the land of probability's sway,
I'll follow the numbers, come what may,
My heart beats for Bayes' theorem's might,
Guiding me through the uncertainty of night."


In [14]:
from llama_index.core import VectorStoreIndex
from pprint import pprint
index = VectorStoreIndex.from_documents(docs_bayes)
query_engine = index.as_chat_engine(similarity_top_k=5)

In [15]:
response = query_engine.query("Explain cosine similarity, using info from the book on Data Mining")

In [94]:
print(response)

Cosine similarity is a measure of the similarity between two vectors, often used in machine learning and data analysis. It is calculated as the dot product of the two vectors divided by the product of their magnitudes. In other words, it measures the angle between the two vectors, with a smaller angle indicating a higher similarity.

In the context of text analysis, cosine similarity is often used to measure the similarity between documents or sentences. It is particularly useful when dealing with high-dimensional data, such as text documents, where traditional measures of similarity may not be effective.

For example, in the context of search engines, cosine similarity can be used to rank search results based on their relevance to the query. The search engine can calculate the cosine similarity between the query and each document in the index, and then rank the documents based on their similarity scores.

In addition to search engines, cosine similarity has many other applications in 

In [16]:
def GetSources(response):
  for node in response.source_nodes:
      # Access the TextNode object directly
      text_node = node.node

      # Assuming metadata is stored within the TextNode's metadata
      source = text_node.metadata.get('file_name') # Access metadata using .metadata.get()
      page = text_node.metadata.get('page_label')  # Access metadata using .metadata.get()

      print(f"Source: {source[:30]}...")
      print(f"Page: {page}")

GetSources(response)

Source: The Theory That Would Not Die ...
Page: 278
Source: The Theory That Would Not Die ...
Page: 275
Source: The Theory That Would Not Die ...
Page: 274
Source: The Theory That Would Not Die ...
Page: 277
Source: The Theory That Would Not Die ...
Page: 264


In [17]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_bayes)
summary_engine = summary_index.as_query_engine()

In [18]:
response = summary_engine.query(
    """Given info from the provided sources, analyze how Bayesian probability transformed
    from a controversial mathematical concept to a critical tool in military intelligence"""
)

KeyboardInterrupt: 

In [69]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [91]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True, llm=llm_70b
)

response = query_engine.query(
    # "Tell me about the specific details about Alan Turing's cryptographic methods at Bletchley Park"
    "Summarize the provided sources and analyze how Bayesian probability transformed from a controversial mathematical concept to a critical tool in military intelligence"
)

Selecting query engine 1: The question asks to summarize the provided sources, which implies condensing a large amount of information into a shorter form, making option 2 the most relevant choice..


In [92]:
print(response)

The development of Bayesian probability from a controversial concept to a critical tool in military intelligence is a story of gradual acceptance and adaptation. Initially, Bayesian methods were met with skepticism and controversy, particularly in the 1950s and 1960s. However, as the concept evolved and its applications became more widespread, it gained recognition as a powerful tool for decision-making and uncertainty quantification.

One of the key factors contributing to the transformation of Bayesian probability was its adoption by the military. The National Research Council of the National Academy of Sciences strongly urged the U.S. Army to use Bayesian methods for testing weapons systems, specifically the Stryker family of light, armored vehicles. This marked a significant turning point, as the military's endorsement of Bayesian methods helped to legitimize the concept and demonstrate its practical applications.

Another important factor was the work of pioneers like Dennis Lindl

In [71]:
pprint(response)

Response(response="Alan Turing's cryptographic methods at Bletchley Park "
                  'involved the development of a manual method for reducing '
                  'the burden on the bombes, which he nicknamed Banburismus. '
                  'This method was a highly labor-intensive, Bayesian system '
                  'that used a unit of measurement called a "ban" to '
                  'systematize his hunches and compare their probabilities. '
                  'The ban was defined as "about the smallest change in weight '
                  'of evidence that is directly perceptible to human '
                  'intuition" and represented odds of 10 to 1 in favor of a '
                  "guess. Turing's method involved using cribs, which were "
                  'German words predicted to occur in the plain-text, to '
                  "identify the settings for 2 of Enigma's 3 wheels and reduce "
                  'the number of wheel settings to be tested on the bombes '


In [76]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about the Kendrick and Drake beef."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
response = chat_engine.chat(
    "Tell me about the songs Drake released in the beef."
)
print(str(response))

Condensed question: Tell me about the songs Drake released in the beef.
Context: page_label: 219
file_path: upload/The Theory That Would Not Die How Bayes Rule Cracked the Enigma Code, Hunted Down Russian Submarines, and Emerged Triumphant from Two Centuries of Controversy by Sharon Bertsch McGrayne.pdf

bombs and radioactivity leaked out only after an American sergeant shouted
to the first reporter on the scene, “Hey, buddy, can you speak Spanish?”
“Sure.”
“Well, tell that peasant over there to get out of that field, for God’s sake. I
can’t make him understand a damned thing. There’s radioactivity there and
we’ve got to keep the people cleared out.”
A public relations catastrophe was in the making. This was the first
accident involving the widespread dispersal of radioactive material and the
first that attracted widespread, highly critical scrutiny from the world’s
media. Within three days, reporters in Palomares knew that a nuclear bomb
was missing, but six weeks passed before the U.

In [19]:
import json
from typing import Sequence, List

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent

import nest_asyncio

nest_asyncio.apply()

In [20]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)

In [21]:
agent = ReActAgent.from_tools(
    [multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=llm_70b,
    verbose=True,
)

In [22]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

> Running step e2b5f556-8708-4255-8175-48d679cd2844. Step input: What is (121 + 2) * 5?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question. I will first calculate 121 + 2.
Action: add
Action Input: {'a': 121, 'b': 2}
Observation: 123
> Running step 72746494-b86a-4aa3-abfb-e8f64f57d386. Step input: None
Thought: Now that I have the result of 121 + 2, I can use it to calculate (121 + 2) * 5.
Action: multiply
Action Input: {'a': 123, 'b': 5}
Observation: 615
> Running step b62c06b6-9636-4c90-8311-ea72f89bd0da. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: (121 + 2) * 5 is 615.
(121 + 2) * 5 is 615.


In [23]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [24]:
bayes_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="bayes_search",
        description="Useful for searching over Thomas Bayes' life.",
    ),
)

wwii_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="wwii_search",
        description="Useful for searching over WWII related information.",
    ),
)

query_engine_tools = [bayes_tool, wwii_tool]

In [25]:
agent = ReActAgent.from_tools(
    query_engine_tools,  ## TODO: define query tools
    llm=llm_70b,
    verbose=True,
)

In [26]:
response = agent.chat("Tell me about Thomas Bayse early work on theology"
" and how this is related to Bayes theorem")
print(str(response))

> Running step 13791f58-f35b-4d92-8930-302e4e5acb80. Step input: Tell me about Thomas Bayse early work on theology and how this is related to Bayes theorem
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about Thomas Bayes' early work on theology and its relation to Bayes' theorem.
Action: bayes_search
Action Input: {'input': 'Thomas Bayes early work theology Bayes theorem'}
Observation: Thomas Bayes' early work in theology was focused on reconciling the presence of evil with God's presumed beneficence. He wrote a pamphlet in 1731 declaring that God gives people "the greatest happiness of which they are capable."
> Running step f2b9e531-aeb5-4835-a1d6-2b7a09258263. Step input: None
Thought: I have some information about Thomas Bayes' early work in theology, but I need more information about how this is related to Bayes' theorem. I'll use the tool again to find more information.
Action: bayes_search
Action Input: {'input': 'Thom

In [27]:
pprint(response)

AgentChatResponse(response="Thomas Bayes' early work in theology, particularly "
                           'his efforts to reconcile the presence of evil with '
                           "God's presumed beneficence, had a profound "
                           "influence on his development of Bayes' theorem.",
                  sources=[ToolOutput(content='Thomas Bayes\' early work in theology was focused on reconciling the presence of evil with God\'s presumed beneficence. He wrote a pamphlet in 1731 declaring that God gives people "the greatest happiness of which they are capable."', tool_name='bayes_search', raw_input={'input': 'Thomas Bayes early work theology Bayes theorem'}, raw_output=Response(response='Thomas Bayes\' early work in theology was focused on reconciling the presence of evil with God\'s presumed beneficence. He wrote a pamphlet in 1731 declaring that God gives people "the greatest happiness of which they are capable."', source_nodes=[NodeWithScore(node=TextNode(id_

In [87]:
response = agent.chat("Tell me about interaction between Alan Turing and Claude Shannon"
                      " and how this impact Shannon's work on information theory"
)
print(str(response))

> Running step b390ac4e-9049-44b8-a015-e3d0c1e11b72. Step input: Tell me about interaction between Alan Turing and Claude Shannon and how this impact Shannon's work on information theory
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about WWII related information, specifically about Alan Turing and Claude Shannon's interaction.
Action: wwii_search
Action Input: {'input': 'Alan Turing and Claude Shannon interaction information theory'}
Observation: When Turing visited Bell Labs, he and Shannon likely discussed the SigSaly voice scrambler, which Shannon was developing.
> Running step c9ae03bb-1a34-4da3-9823-230d520fc87a. Step input: None
Thought: I have some information about the interaction between Alan Turing and Claude Shannon, but I need more information about how this interaction impacted Shannon's work on information theory.
Action: wwii_search
Action Input: {'input': 'Turing and Shannon discussion on SigSaly voice scramb

In [88]:
response = agent.chat("Tell me about how Bayes theorem is used in Midway Battle"
)
print(str(response))

> Running step 51df88d9-d163-488a-992e-8e3e9b0beb53. Step input: Tell me about how Bayes theorem is used in Midway Battle
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about how Bayes theorem is used in the Midway Battle.
Action: wwii_search
Action Input: {'input': 'Bayes theorem Midway Battle'}
Observation: There is no mention of the Midway Battle in the provided context.
> Running step 058b966a-9149-4b7a-86a0-e70f7fed819c. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about how Bayes theorem is used in the Midway Battle. Since the previous search did not provide relevant information, I will try to search again with more specific keywords.
Action: wwii_search
Action Input: {'input': 'Midway Battle codebreaking'}
Observation: The codebreaking efforts at Bletchley Park were crucial in the Allied victory, but the Midway Battle codebreaking is not ment